# SynthKernel

SynthKernel is a software architecture in TypeScript that enforces type safety, modularity and composability as well as making the APP easily extensible. It's rather a philosophy, a methodology and a set of standards than a bunch of static code.

SynthKernel leverages strengths in _Object-Oriented Programming_, _TypeScript Type System and its Dynamic Nature_, _Facade Pattern_, and _Inversion of Control Principle_. It is born to fundamentally solve the problem of software unmaintainability by enforcing strict naming, file system, separation of concern and module composition standards without stifling logic flexibility. It also provides detailed patterns and templates on how to architect an application.

## Architecture Overview

SynthKernel does impose very strict separation-of-concern disciplines, which ensures later maintainability and ease of codebase understanding once your understand SynthKernel. With separation, any project adopting SynthKernel can be easily visualized via a tree diagram and fit into file system constrains. Here's an example structure.

```mermaid
flowchart TD
    loader[loader]
    mod1[module]
    mod2[module]
    mod3[module]
    mod4[module]
    mod5[module]
    sub1[sub-loader]

    %% Connections
    loader --> mod1
    loader --> mod2
    loader --> sub1

    sub1 --> mod3
    sub1 --> mod4
    sub1 --> mod5
```

From the diagram, you can easily observe three types of nodes:

- loader: stem of the tree
- module: leaf of the tree
- sub-loader: branch of the tree

Loaders serve for three purposes - lifecycle manager of children modules, orchestrator of children-contributed types and the facade between the loader consumer and app's code. Loader itself does not do anything related to the app logic in most cases.

Modules are responsible for the app logic, they register lifecycle hooks, orchestrate types, wire each other via dependency injection, and augment their loader.

Sub-loaders can naturally emerge when your application has grown to a degree of complexity, where your find a module has been bloated too much. You can make the loader a new loader-module hierarchy to orchestrate the task.

## Mechanisms

### Inter-Module Communication

Complex application often require tight collaboration between different functionalities, this often leads to tight coupling. SynthKernel solves this via **module level dependency injection**.

SynthKernel modules are mostly singleton, to achieve inter-module communication, modules need to inject other modules. That is, modules are both service providers and subjects that is being injected to. All dependency injection happens autonomously at module level, while the loader simply registers all direct modules into a container that is passed to modules via constructor injection.

Combined with **module defined hooks**, dependency injection ensures loose coupling even in the most integrated functions, and makes any module can reach any part of the application like a traditional monolith. Since modules can only access modules explicitly registered in the container (often sibling modules and explicitly defined by the loader), this pattern can also ensure explicit dependency akin to React's unidirectional data flow.

### Lifecycle Hooks

Different applications require different lifecycle strategies. For example, a simple application may only have global construction and disposal, while a highly composable software may need to start and stop any module at any time. This generates lifecycle management requirements that cannot be managed in a single module. Loaders should manage in a collective manner.

Lifecycle hooks should be defined in the loader as simple subscription hooks like `onDispose` and `onModuleRestart`. The hooks should be passed to modules via constructor injection and then consumed by modules according to their needs.

### Base Module

Modules need a starting template so that they can implement their functions freely. A base module, often in companion with a loader, is inherited by all modules of the loader. The module should do some fixed tasks like receive DI container and lifecycle hooks passed from the loader.

More importantly, base modules serve as interfaces of type orchestration and augmentation declaration. And it also takes the role of type re-interpretation for some methods & properties of orchestrated types. This part will be elaborated below in [Type Orchestration](#type-orchestration) and [Augmentation](#augmentation).

### Type Orchestration

Highly modularized and extensible application often embodies poorly managed types, SynthKernel breaks the hell by declarative type orchestration via advanced generics.

For example, it is common for a module to define some configurable fields that can change the module's runtime behavior. Traditional application often uses a centralized configuration file to store these fields, which breaks the modularity by forcing modules to depend on a centralized configuration file. In SynthKernel, each module contribute their own atomic slices of configuration declaratively to the loader. Then the it merges these slices into a single object.

For example, module A can declare a configuration field like this:

```TypeScript
interface Config {
	a?: number;
}
```

And module B has config like this:

```TypeScript
interface Config {
	b: boolean;
}
```

The final config will be:

```TypeScript
interface Config {
	a?: number;
	b: boolean;
}
```

Moreover, it's common for the loader to pass some methods or properties that needs type mapping to the modules. For example, the loader receives the user defined configuration and then pass it to the modules. It cannot pass the config in exact type since the it is defined in generics. This creates a problem that the modules cannot receive the precise type even it they defined it themselves. To solve this, the type is intercepted by the base module, it augments the type using the module's type declaration. Then everything become accurately typed while is still modular and pluggable.

### Augmentation

Augmentation is another key component powered by SynthKernel's type orchestration capability. Without augmentation, the consumers need to access the DI container for APP functionalities while the loader can only do some basic lifecycle works. Augmentation allows modules to inject methods and properties back to the loader with full type safety, making it become a facade that encapsules app logic. The consumer can have accurate type hint as well as real runtime logic **on the loader** that comes from modules.

### Sub-Loader

Sub-loader is a combination of loader and module. Itself is designed to be a module of its parent loader, while it has grown to a degree of complexity that a new loader-module hierarchy is needed to orchestrate its functionality.

To create a sub-loader, simply create a new loader class as usual, then create another module class that instantiates the loader class. To put it another way, a sub-loader is a module plus a loader.

### Unidirectional Logic Flow

Sub-loader provides a modal of scalability by delegating growing complex logic to new self-sustaining units. However, this creates a hierarchy which comes with inter-hierarchal communication challenges. Bad design can still lead to unmaintainable code. Inspired by React, a logic flow standard is introduced to help build clear and explicit dependency relationships:

1. Each hierarchy is a self-sustaining unit with its isolated DI container.
2. Each module can only access its siblings and communicate with its parent.
3. To use a module in a higher hierarchy, it must be obtained from the parent DI container and provided into the child DI container by the sub-loader, this is exactly the same philosophy of setter and getter functions - they seem to be redundant, but they are helping you to keep the logic clean.
4. To subscribe module-defined hooks in a higher hierarchy, prefer to augment the sub-loader which is responsible for the subscription. Avoid directly subscribing to higher hooks in the module.

Above can be summarized as a unidirectional flow: **Raw down, fine up**.

What's down:

1. raw input
2. raw module logic

What's up:

1. selected methods and properties augmented to the loader
2. module-declared types to orchestrate

## End-to-End Example

Here is an end-to-end example of a application using SynthKernel. We are going to build a simple backend logger with alerting capability.

Let's call the logger `PolisAlert`, it will have two modules:

1. `CoreLogging`: the logging module that provides configurable and classified logging capability.
2. `AlertDispatch`: the alerting module that validates alerts and dispatches them to external services, like an email API.

The architecture of `PolisAlert` can be show as a very simple tree:

```mermaid
flowchart TD
    loader[PolisAlert Loader]
    mod1[CoreLogging Module]
    mod2[AlertDispatch Module]

    %% Connections
    loader --> mod1
    loader --> mod2
```

### Let's Code

Firstly, we need to import what we will use next. All we need is a dependency injection library, here we choose a super lightweight solution called `@needle-di/core`.

In [14]:
import { Container } from '@needle-di/core';

console.log('Needle DI imported!');

Needle DI imported!


Then we need to copy some boilerplate code. The following code is for type orchestration. The implementation of SynthKernel does vary greatly across different requirements. And the type orchestration is the rare fixed code:

In [15]:
type General = any;
type GeneralArray = ReadonlyArray<General>;
type GeneralObject = object;
type GeneralConstructor = new (...args: General[]) => General;

type UnionToIntersection<U> =
	(U extends General ? (k: U) => void : never) extends (
		k: infer I,
	) => void ? I
		: never;

type GeneralModuleInput =
	| ReadonlyArray<GeneralConstructor>
	| ReadonlyArray<GeneralObject>;

type ModuleInput<T extends GeneralConstructor> =
	| ReadonlyArray<T>
	| ReadonlyArray<InstanceType<T>>;

type Instances<T extends GeneralModuleInput> = T extends
	ReadonlyArray<GeneralConstructor> ? InstanceType<T[number]> : T[number];

type Orchestratable<
	T extends GeneralModuleInput,
	K extends keyof Instances<T>,
> = UnionToIntersection<Instances<T>[K]>;

console.log('Types loaded!');

Types loaded!


Feel overwhelmed by the type orchestration? You only need to copy and paste!

Now you can test the orchestration result by hovering over the type `Orchestrated`:

In [16]:
type Object1 = {
	orchestrate: {
		a: string;
		b: number;
	};
};

type Object2 = {
	orchestrate: {
		c?: boolean;
		d: () => void;
	};
};

type Orchestrated = Orchestratable<[Object1, Object2], 'orchestrate'>;

/*
Orchestrated will be the type:
{
    a: string;
    b: number;
    c?: boolean;
    d: HTMLDivElement;
}
*/

console.log('Actually running this cell has no real effect.');

Actually running this cell has no real effect.


We also need a tiny utility function to make a hook that can be subscribed to. This function creates another function with properties that store and manipulate the subscription list.

In [17]:
type MatchingFunc<Args extends GeneralArray> = (...args: Args) => unknown;
type Hook<Args extends GeneralArray = []> = {
	(...args: Args): void;
	subs: Set<MatchingFunc<Args>>;
	subscribe(callback: MatchingFunc<Args>): void;
	unsubscribe(callback: MatchingFunc<Args>): void;
};

/**
 * A quick function to create a hook that can be subscribed to and unsubscribed from.
 * Pass your arguments as the type parameter
 * @example const hook = makeHook(true); // create a hook that runs subscriptions in reverse order
 */
function makeHook<Args extends GeneralArray = []>(reverse: boolean = false) {
	const result: Hook<Args> = (...args: Args) => {
		if (reverse) {
			const items = Array.from(result.subs).reverse();
			items.forEach((callback) => {
				callback(...args);
			});
		} else {
			result.subs.forEach((callback) => {
				callback(...args);
			});
		}
	};
	result.subs = new Set();
	result.subscribe = (callback: MatchingFunc<Args>) => {
		result.subs.add(callback);
	};
	result.unsubscribe = (callback: MatchingFunc<Args>) => {
		result.subs.delete(callback);
	};
	return result;
}

console.log('Hook maker loaded!');

Hook maker loaded!


How we can focus on the project functionality. Let's start with making some options for the user to configure `PolisAlert`. These options come to reflect global state so it's improper to declare them in a module, we declare them on the top of everything. Here we have two base options:

- `appName`: the name of the app
- `debug`: enables debug mode

In [18]:
interface BaseOptions {
	appName: string;
	debug?: boolean;
}

console.log('Base options declared!');

Base options declared!


Then let's define the base module so that modules have a common interface. The base module will serve for purposes:

- Receive the module's orchestration declarations, which will be based on the base orchestrations we defined before (currently `BaseOptions`).
- Receive constructor injections from the loader and augment their types according to type declarations.

In [19]:
class BaseModule<
	O extends BaseOptions = BaseOptions,
	A extends GeneralObject = {},
> {
	declare private static readonly _BaseModuleBrand: unique symbol; // this is a hint fot TypeScript to only identify objects that truly extends BaseModule can pass the type check
	declare _Augmentation: A;

	options: O;

	onStart: Hook['subscribe'];
	onDispose: Hook['subscribe'];

	container: Container;
	augment: (aug: A) => void;
	constructor(
		container: Container,
		options: GeneralObject,
		onStart: Hook,
		onDispose: Hook,
		augment: (aug: A) => void,
	) {
		this.container = container;
		this.augment = augment;
		// we assign the above two lines for Node.js compatibility. If you have a TypeScript compiler you can remove them and simply write in the constructor parameters:
		// protected container: Container,
		// protected augment: (aug: A) => void,

		this.options = options as O;

		this.onStart = onStart.subscribe;
		this.onDispose = onDispose.subscribe;
	}
}

console.log('Base module loaded!');

Base module loaded!


Then we will make some types for later uses, including:

- general module types that will be used in the loader and modules
- orchestration of options and augmentation types

In [20]:
type GranularModuleInput = ModuleInput<GeneralModuleCtor>;
type GeneralModuleCtor = typeof BaseModule<General, General>;
type BaseArgs = ConstructorParameters<GeneralModuleCtor>;

// the granular orchestration types
type OptionsOrchestratable<M extends GranularModuleInput> = Orchestratable<
	M,
	'options'
>;
type AugmentationOrchestratable<M extends GranularModuleInput> = Orchestratable<
	M,
	'_Augmentation'
>;

console.log('Additional types loaded!');

Additional types loaded!


Everything is prepared! We can proceed with the logging module. It needs to provide a logging method that determines whether to log according to the consumer's config, record down all the logs, and provide a way to retrieve them.

So it will contribute two options:

- `logLevel`: The log level above which the log will be shown in the console.
- `maxLogs`: The maximum number of logs to be stored, if the number of logs exceeds this number, the oldest logs will be deleted.

Then it needs to augment the `log` method and `logs` getter to the loader for easy access.

In [21]:
const LEVELS = { DEBUG: 0, INFO: 1, WARN: 2, ERROR: 3 } as const;
type Level = keyof typeof LEVELS;

interface LoggingOptions extends BaseOptions {
	logLevel: Level;
	maxLogs?: number;
}
interface LoggingAugmentation {
	log: CoreLogging['log'];
	logs: ReadonlyArray<LogEntry>;
}

interface LogEntry {
	timestamp: number;
	level: string;
	message: string;
}

class CoreLogging extends BaseModule<LoggingOptions, LoggingAugmentation> {
	private _logs: LogEntry[] = [];

	constructor(...args: BaseArgs) {
		super(...args);
		const self = this;
		this.augment({
			log: this.log,
			get logs() {
				return Object.freeze([...self._logs]);
			},
		});
		this.onStart(() => {
			this.log('INFO', 'CoreLogging initialized');
		});
		this.onDispose(() => {
			this.log('INFO', 'CoreLogging disposed');
			this._logs = [];
		});
	}

	log = (level: Level, message: string) => {
		const currentLevel = LEVELS[level];
		const minLevel = LEVELS[this.options.logLevel] ?? 0;
		if (currentLevel < minLevel) return; // Skip logging if below threshold
		const entry: LogEntry = {
			timestamp: Date.now(),
			level,
			message,
		};
		const maxLogs = this.options.maxLogs ?? 1000;
		if (this._logs.length >= maxLogs) this._logs.shift();
		this._logs.push(entry);

		// Always print if debug mode is forced in base options, otherwise respect level
		if (this.options.debug || currentLevel >= minLevel) {
			console.log(`[${level}] ${message}`);
		}
	};
}

console.log('CoreLogging loaded!');

CoreLogging loaded!


Based on the logging module, we can now design the alerting module. This module will be responsible for sending alerts to external services and record the alerts via the log module.

Since we cannot truly use an external service in a Jupyter Notebook, we will use a mock service that simulates an asynchronous trigger. We also allows the user to specify the max and min length of the alert to mimic the real alert service volume.

Moreover, the module will contribute a method `dispatchAlert` to the loader facade.

In [22]:
interface AlertOptions extends BaseOptions {
	minMessageLength: number;
	maxMessageLength: number;
}

interface AlertAugmentation {
	dispatchAlert: AlertDispatch['dispatchAlert'];
}

class AlertDispatch extends BaseModule<AlertOptions, AlertAugmentation> {
	private logging: CoreLogging;

	constructor(...args: BaseArgs) {
		super(...args);
		this.augment({ dispatchAlert: this.dispatchAlert });
		this.logging = this.container.get(CoreLogging);
		this.onStart(() => {
			this.logging.log('INFO', 'AlertDispatch initialized');
		});
		this.onDispose(() => {
			this.logging.log('INFO', 'AlertDispatch disposed');
		});
	}

	dispatchAlert = async (message: string): Promise<boolean> => {
		this.logging.log('INFO', `Attempted dispatch: "${message}"`);
		const { minMessageLength, maxMessageLength } = this.options;
		if (message.length < minMessageLength) {
			this.logging.log(
				'ERROR',
				`Validation failed: message too short (min: ${minMessageLength})`,
			);
			return false;
		}
		if (message.length > maxMessageLength) {
			this.logging.log(
				'ERROR',
				`Validation failed: message too long (max: ${maxMessageLength})`,
			);
			return false;
		}

		await this.connectAlertService(message);
		return true;
	};

	private connectAlertService = async (alert: string) => {
		this.logging.log('INFO', `Dispatched: "${alert}"`);

		// Simulate async connection to alerting service, like an email api
		await new Promise((resolve) => setTimeout(resolve, 10));
	};
}

console.log('AlertDispatch loaded!');

AlertDispatch loaded!


All modules needed are implemented. The final step is to put them together and design the loader. Firstly, we gather all the orchestration results, you can hover above the types to see the orchestration definition.

In [23]:
const allModules = [CoreLogging, AlertDispatch];
type AllModules = typeof allModules;

// final orchestrated types
type AllOptions = OptionsOrchestratable<AllModules>;
type AllAugmentation = AugmentationOrchestratable<AllModules>;

console.log('Orchestration finished!');

Orchestration finished!


The loader needs to provide all of the functionalities below:

- application lifecycle hooks
- receive configuration
- being augmented to behave as a facade between the application and the consumer
- DI container that registers all of the modules

In [24]:
class Loader {
	private onDispose = makeHook(true);
	private onStart = makeHook();

	options: AllOptions;

	container: Container;

	private augment = (aug: GeneralObject) => {
		const descriptors = Object.getOwnPropertyDescriptors(aug);
		Object.defineProperties(this, descriptors);
	};

	constructor(options: AllOptions) {
		this.container = new Container();
		this.options = options;

		const bind = (Module: GeneralModuleCtor) => {
			this.container.bind({
				provide: Module,
				useFactory: () =>
					new Module(
						this.container,
						this.options,
						this.onStart,
						this.onDispose,
						this.augment,
					),
			});
		};
		allModules.forEach(bind);
		allModules.forEach((Module: GeneralModuleCtor) => {
			this.container.get(Module);
		});
		this.onStart();
	}

	dispose = () => {
		this.onDispose();
		this.container.unbindAll();
	};
}

console.log('Loader loaded!');

Loader loaded!


It's not enough if we stop at the loader, since although the augmentation has injected into the loader, TypeScript doesn't know about it in the types of the loader.

The final step is to tell TypeScript about the augmentation.

In [25]:
type LoaderType = new (
	...args: ConstructorParameters<typeof Loader>
) => Loader & AllAugmentation;

// not every type assertion harms type safety, this one enhances type safety
const PolisAlert = Loader as LoaderType;

console.log('PolisAlert augmented!');

PolisAlert augmented!


We are all done! Let's test it out!

### Final Result

Let us create a consumer script that uses `PolisAlert` to log and alert something:

In [26]:
// all options naturally orchestrated here with correct types
const app = new PolisAlert({
	appName: 'PolisAlert',
	debug: true,
	logLevel: 'DEBUG',
	maxLogs: 500,
	minMessageLength: 1,
	maxMessageLength: 280,
});

// type-safe access to augmented methods
app.log('INFO', 'Application started');

const success = await app.dispatchAlert('Hello Polis');
console.log('Dispatch result:', success);

// access orchestrated state
console.log('Audit trail:', app.logs);

// cleanup
app.dispose();

[INFO] CoreLogging initialized
[INFO] AlertDispatch initialized
[INFO] Application started
[INFO] Attempted dispatch: "Hello Polis"
[INFO] Dispatched: "Hello Polis"
Dispatch result: true
Audit trail: [
  {
    timestamp: 1771683816299,
    level: "INFO",
    message: "CoreLogging initialized"
  },
  {
    timestamp: 1771683816299,
    level: "INFO",
    message: "AlertDispatch initialized"
  },
  {
    timestamp: 1771683816299,
    level: "INFO",
    message: "Application started"
  },
  {
    timestamp: 1771683816299,
    level: "INFO",
    message: 'Attempted dispatch: "Hello Polis"'
  },
  {
    timestamp: 1771683816299,
    level: "INFO",
    message: 'Dispatched: "Hello Polis"'
  }
]
[INFO] AlertDispatch disposed
[INFO] CoreLogging disposed


Everything works as if you were using a monolith! Despite its highly modular nature.

You can adjust the options or log and alert more in the cell above to see what will happen. You can even remove `AlertDispatch` module from `allModules`, the logger will still work fine, you simply won't use any alerts.

## Final Words

Isn't SynthKernel awesome? You've just seen a demo of how modules can work seamlessly together and removing a module even doesn't break anything. Everything is structured and clear, no more "spaghetti" created.

Maybe you'll say "Ah, I'd like to make `PolisAlert` just a monolith within a single file. There's too much boilerplate to do that in your way".

Yes, absolutely. For small scripts, `SynthKernel` is an overkill. But `PolisAlert` is a simple example whose scale is deliberately designed to be small. In real production, when you have tens of modules with 1000+ lines of code, you'll find how `SynthKernel` can make your coding much easier.